<a href="https://colab.research.google.com/github/GabrielData21/ETL-com-PySpark/blob/main/ETL_com_Python_e_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de ETL com Python (PySpark)

**Descrição da atividade**

O objetivo desse projeto é desenvolver um processo de ETL utilizando liguagem Python e o framework PySpark, considerando 3 estágios de transformação dos dados: raw, staging e refined. Foram utilizados os dados do ENEM 2020 para o exercício desse projeto.


Usei o chatGPT pra conseguir alguns comandos que vão ajudar na execução do ETL.

## Instalando PySpark

A partir da versão 3.4.0 é possível realizar a instalação do PySpark no Google Colab de forma isolada. Em versões anteriores era necessário instalar o ambiente com Hadoop, java e outros módulos.

In [1]:
# Instalando o PySpark
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=e4f8d85849fd280d75d85316c287f85e496bb7bfc556e987b068060a3214c775
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
# Iniciando sessão do PySpark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL-enem").getOrCreate()
spark

In [62]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# Organizando o ambiente para os arquivos do ETL

Solicitei ao chatGPT os comandos adequados para criar as pastas onde vamos guardar os arquivos do ETL em seus 3 estágios: raw, staging e refined. Os comando são os mesmos utilizados no Linux Shell para criação de diretórios.

In [3]:
# Criando a pasta mestre do projeto
!mkdir enem_2020/

In [4]:
# Criando as pastas dos estágios do ETL
!mkdir enem_2020/1.raw/ enem_2020/2.staging/ enem_2020/3.refined/

## Aquisição de dados ENEM

### Download e pré processamento dos dados

In [5]:
# Captação dos dados do ENEM 2020
!wget https://download.inep.gov.br/microdados/microdados_enem_2020.zip --no-check-certificate

--2023-09-21 16:09:26--  https://download.inep.gov.br/microdados/microdados_enem_2020.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 620776982 (592M) [application/zip]
Saving to: ‘microdados_enem_2020.zip’

microdados_enem_202 100%[===================>] 592.02M   444KB/s    in 24m 8s  

2023-09-21 16:33:35 (419 KB/s) - ‘microdados_enem_2020.zip’ saved [620776982/620776982]



In [6]:
# Mostrando os arquivos baixados
!ls -l

total 606240
drwxr-xr-x 5 root root      4096 Sep 21 16:09 enem_2020
-rw-r--r-- 1 root root 620776982 Feb 16  2022 microdados_enem_2020.zip
drwxr-xr-x 1 root root      4096 Sep 19 13:44 sample_data


Os arquivos do ENEM são recebidos em formato compactado, para fazer a descompactação existem duas opções: usando comandos linux shell (através do unzip) ou usando o pacote zipfile do Python. Como este exercício é uma prática em Python e PySpark, usei o pacote zipfile. As recomendações para descompactação foram feitas pelo chatGPT.

In [7]:
# Importando pacote para descompactar o arquivo ENEM 2020
import zipfile

In [8]:
# Medindo o tempo de descompactação
%%time

with zipfile.ZipFile('microdados_enem_2020.zip', 'r') as zip_ref:
    zip_ref.extractall('enem_2020')

CPU times: user 15.5 s, sys: 8.64 s, total: 24.2 s
Wall time: 31.8 s


Outro passo necessário é organizar os arquivos da descompactação, usei apenas o arquivo principal para a transformação. Por isso ele será movido para a pasta /raw.

Mover arquivos entre pastas usando comandos linux shell é bem simples através do comando mv. Novamente, para manter o treinamento voltado para Python, usei o pacote shutil para realizar a movimentação e seus comandos foram passados pelo chatGPT.

In [9]:
import shutil

In [11]:
"""
O código comentado abaixo foi passado pelo chat GPT,
porém não é suficiente para realizar a movimentação da forma necessária.
A documentação foi consultada para conseguir os detalhes faltantes.
"""
# shutil.move('arquivo.txt', 'nova_pasta/')

# Atribuuindo os endereços dos dados a variáveis
source = "/content/enem_2020/DADOS/MICRODADOS_ENEM_2020.csv"
destination = "/content/enem_2020/1.raw/MICRODADOS_ENEM_2020.csv"

# Realizando a movimentação
shutil.move(source, destination)

'/content/enem_2020/1.raw/MICRODADOS_ENEM_2020.csv'

In [63]:
%%time
import pyspark.sql.functions

dados_enem = spark.read.csv('/content/enem_2020/1.raw/MICRODADOS_ENEM_2020.csv', header= True, sep=';', encoding='ISO-8859-1')
dados_enem.show(5)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [14]:
"""
Verificando as colunas do DataFrame, avaliando junto à documetação do dicionário dos dados
sobre o conteúdo, formato...
"""
dados_enem.columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

# Transformação dos dados

Nessa etapa realizarei a seleção dos dados relevantes para uma análise posterior, essas informações são acordadas com a área da análises que precisa desses dados.
Para esse projeto, considerei que a análise na ponta realizará estudos com os dados de:
* notas de cada categoria de conhecimento do ENEM;
* nota geral da redação;
* nota geral do ENEM;
* faixa etária;
* análise demográfica a nível de estado (de acordo com local de aplicação da prova);
* cor/raça;
* treineiro;
* quantidade de faltantes;
* sexo;
* situação de conclusão do ensino médio;
* Tipo de instituição que concluiu ou concluirá o Ensino Médio;
* tipo de escola no ensino médio.

In [15]:
"""
Para realizar o tratamento de forma mais eficiente, o arquivo será transformado em Parquet.
Segundo chat GPT, podemos atribuir o destino do arquivo em uma variável para usar na função
de salvar o arquivo no formato desejado.
"""

# Caminho para salvar o arquivo Parquet
parquet_path = "/content/enem_2020/refined/MICRODADOS_ENEM_2020.parquet"

# Salvar o DataFrame como arquivo Parquet
dados_enem.write.mode("overwrite").parquet(parquet_path)

In [64]:
%%time

# Pra ler o arquivo Parquet pode usar a mesma variável criada para alocar o aquivo na célula anterior
enem_parquet = spark.read.parquet(parquet_path)
enem_parquet.show(5)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

Parte do processo de ETL é a seleção das colunas que contém os dados relevantes para análises posteriores pela equipe de Analytics.

Em um dataset que contém muitas colunas como esse, para manter o código limpo é interessante colocar as colunas selecionadas (seja para remoção ou que serão mantidas) em uma variável antes de colocar na função. Aqui, essa seleção foi realizada utilizando o documento de dicionário de dados mantendo o perfil de análise em mente.

In [20]:
# Imprimindo as colunas em uma lista
enem_parquet.columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

In [26]:
# Selecionando as colunas para a área de Analytics
dropcols = ('TP_ESTADO_CIVIL',
 'TP_NACIONALIDADE',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009',
 'Q010',
 'Q011',
 'Q012',
 'Q013',
 'Q014',
 'Q015',
 'Q016',
 'Q017',
 'Q018',
 'Q019',
 'Q020',
 'Q021',
 'Q022',
 'Q023',
 'Q024',
 'Q025')

In [27]:
enem_parquet = enem_parquet.drop(*dropcols)
enem_parquet.show(5)

+------------+------+---------------+-------+-----------+---------------+---------------+---------+---------+------------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|TP_LINGUA|TP_STATUS_REDACAO|NU_NOTA_COMP1|NU_NOTA_COMP2|NU_NOTA_COMP3|NU_NOTA_COMP4|NU_NOTA_COMP5|NU_NOTA_REDACAO|
+------------+------+---------------+-------+-----------+---------------+---------------+---------+---------+------------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+-------------+-------------+

In [28]:
# Contando quantos registros tem na tabela
enem_parquet.count()

5783109

Até esse momento realizei uma transformação simples dos dados alterando o formato do arquivo e removendo colunas não essenciais para o trabalho de analytics. A partir de agora trato sobre o formado dos dados, nome de colunas e enriquecimento. Essa transformação final será realizada para colocar os dados no estágio final refined.

In [29]:
# Listando as colunas novamente
enem_parquet.columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO']

Interessante notar que segundo chatGPT só é possível renomear várias colunas seguindo uma estrutura repetitiva como exemplificado abaixo:

df = df.withColumnRenamed("coluna_antiga_1", "nova_coluna_1") \
        .withColumnRenamed("coluna_antiga_2", "nova_coluna_2") \
        .withColumnRenamed("coluna_antiga_3", "nova_coluna_3")

Porém, a partir do PySpark 3.4 que estou usando, foi adicionado suporte para renomear várias colunas com um mesmo comando. Abaixo temos o exemplo de código da documentação:

df = df.withColumnsRenamed({"coluna_antiga_1": "nova_coluna_1", "coluna_antiga_2": "nova_coluna_2", "coluna_antiga_3": "nova_coluna_3"})

In [57]:
enem_refined = enem_parquet.withColumnsRenamed({
 'NU_INSCRICAO':'INSCRICAO',
 'NU_ANO':'ANO',
 'TP_FAIXA_ETARIA':'FAIXA_ETARIA',
 'TP_SEXO':'SEXO',
 'TP_COR_RACA':'COR_RACA',
 'TP_ST_CONCLUSAO':'ST_CONCLUSAO',
 'TP_ANO_CONCLUIU':'ANO_CONCLUIU',
 'TP_ESCOLA':'ESCOLA',
 'TP_ENSINO':'ENSINO',
 'IN_TREINEIRO':'TREINEIRO',
 'SG_UF_PROVA':'UF_PROVA',
 'TP_PRESENCA_CN':'PRE_C_NATURAIS',
 'TP_PRESENCA_CH':'PRE_C_HUMANAS',
 'TP_PRESENCA_LC':'PRE_LINGUAGENS',
 'TP_PRESENCA_MT':'PRE_MATEMATICA',
 'NU_NOTA_CN':'NOTA_C_NATURAIS',
 'NU_NOTA_CH':'NOTA_C_HUMANAS',
 'NU_NOTA_LC':'NOTA_LINGAGENS',
 'NU_NOTA_MT':'NOTA_MATEMATICA',
 'TP_LINGUA':'LINGUA_ESTR',
 'TP_STATUS_REDACAO':'STATUS_REDACAO',
 'NU_NOTA_COMP1':'NOTA_COMP1',
 'NU_NOTA_COMP2':'NOTA_COMP2',
 'NU_NOTA_COMP3':'NOTA_COMP3',
 'NU_NOTA_COMP4':'NOTA_COMP4',
 'NU_NOTA_COMP5':'NOTA_COMP5',
 'NU_NOTA_REDACAO':'NOTA_REDACAO'
 })

In [58]:
enem_refined.show(5)

+------------+----+------------+----+--------+------------+------------+------+------+---------+--------+--------------+-------------+--------------+--------------+---------------+--------------+--------------+---------------+-----------+--------------+----------+----------+----------+----------+----------+------------+
|   INSCRICAO| ANO|FAIXA_ETARIA|SEXO|COR_RACA|ST_CONCLUSAO|ANO_CONCLUIU|ESCOLA|ENSINO|TREINEIRO|UF_PROVA|PRE_C_NATURAIS|PRE_C_HUMANAS|PRE_LINGUAGENS|PRE_MATEMATICA|NOTA_C_NATURAIS|NOTA_C_HUMANAS|NOTA_LINGAGENS|NOTA_MATEMATICA|LINGUA_ESTR|STATUS_REDACAO|NOTA_COMP1|NOTA_COMP2|NOTA_COMP3|NOTA_COMP4|NOTA_COMP5|NOTA_REDACAO|
+------------+----+------------+----+--------+------------+------------+------+------+---------+--------+--------------+-------------+--------------+--------------+---------------+--------------+--------------+---------------+-----------+--------------+----------+----------+----------+----------+----------+------------+
|200006271946|2020|          11|  

Analisando os tipos das variáveis do arquivo, estão todas classificadas como string. Em sua maioria é assim mesmo que vou manter, considerando que são variáveis qualitativas.
Apenas as colunas de nota das provas alterei para números, essas são variáveis quantitativas.

In [59]:
enem_refined.columns

['INSCRICAO',
 'ANO',
 'FAIXA_ETARIA',
 'SEXO',
 'COR_RACA',
 'ST_CONCLUSAO',
 'ANO_CONCLUIU',
 'ESCOLA',
 'ENSINO',
 'TREINEIRO',
 'UF_PROVA',
 'PRE_C_NATURAIS',
 'PRE_C_HUMANAS',
 'PRE_LINGUAGENS',
 'PRE_MATEMATICA',
 'NOTA_C_NATURAIS',
 'NOTA_C_HUMANAS',
 'NOTA_LINGAGENS',
 'NOTA_MATEMATICA',
 'LINGUA_ESTR',
 'STATUS_REDACAO',
 'NOTA_COMP1',
 'NOTA_COMP2',
 'NOTA_COMP3',
 'NOTA_COMP4',
 'NOTA_COMP5',
 'NOTA_REDACAO']

In [46]:
list(enem_refined.schema)

[StructField('INSCRICAO', StringType(), True),
 StructField('ANO', StringType(), True),
 StructField('FAIXA_ETARIA', StringType(), True),
 StructField('SEXO', StringType(), True),
 StructField('COR_RACA', StringType(), True),
 StructField('ST_CONCLUSAO', StringType(), True),
 StructField('ANO_CONCLUIU', StringType(), True),
 StructField('ESCOLA', StringType(), True),
 StructField('ENSINO', StringType(), True),
 StructField('TREINEIRO', StringType(), True),
 StructField('UF_PROVA', StringType(), True),
 StructField('PRE_C_NATURAIS', StringType(), True),
 StructField('PRE_C_HUMANAS', StringType(), True),
 StructField('PRE_LINGUAGENS', StringType(), True),
 StructField('PRE_MATEMATICA', StringType(), True),
 StructField('NOTA_C_NATURAIS', StringType(), True),
 StructField('NOTA_C_HUMANAS', StringType(), True),
 StructField('NOTA_LINGAGENS', StringType(), True),
 StructField('NOTA_MATEMATICA', StringType(), True),
 StructField('LINGUA_ESTR', StringType(), True),
 StructField('STATUS_REDACA

In [51]:
# Importando o pacote types do PySpark para alterar os dados do dataframe
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [60]:
enem_refined = enem_refined.select(
 enem_refined.INSCRICAO.cast(StringType()),
 enem_refined.ANO.cast(StringType()),
 enem_refined.FAIXA_ETARIA.cast(StringType()),
 enem_refined.SEXO.cast(StringType()),
 enem_refined.COR_RACA.cast(StringType()),
 enem_refined.ST_CONCLUSAO.cast(StringType()),
 enem_refined.ANO_CONCLUIU.cast(StringType()),
 enem_refined.ESCOLA.cast(StringType()),
 enem_refined.ENSINO.cast(StringType()),
 enem_refined.TREINEIRO.cast(StringType()),
 enem_refined.UF_PROVA.cast(StringType()),
 enem_refined.PRE_C_NATURAIS.cast(StringType()),
 enem_refined.PRE_C_HUMANAS.cast(StringType()),
 enem_refined.PRE_LINGUAGENS.cast(StringType()),
 enem_refined.PRE_MATEMATICA.cast(StringType()),
 enem_refined.NOTA_C_NATURAIS.cast(IntegerType()),
 enem_refined.NOTA_C_HUMANAS.cast(IntegerType()),
 enem_refined.NOTA_LINGAGENS.cast(IntegerType()),
 enem_refined.NOTA_MATEMATICA.cast(IntegerType()),
 enem_refined.LINGUA_ESTR.cast(StringType()),
 enem_refined.STATUS_REDACAO.cast(StringType()),
 enem_refined.NOTA_COMP1.cast(IntegerType()),
 enem_refined.NOTA_COMP2.cast(IntegerType()),
 enem_refined.NOTA_COMP3.cast(IntegerType()),
 enem_refined.NOTA_COMP4.cast(IntegerType()),
 enem_refined.NOTA_COMP5.cast(IntegerType()),
 enem_refined.NOTA_REDACAO.cast(IntegerType())
)

list(enem_refined.schema)

[StructField('INSCRICAO', StringType(), True),
 StructField('ANO', StringType(), True),
 StructField('FAIXA_ETARIA', StringType(), True),
 StructField('SEXO', StringType(), True),
 StructField('COR_RACA', StringType(), True),
 StructField('ST_CONCLUSAO', StringType(), True),
 StructField('ANO_CONCLUIU', StringType(), True),
 StructField('ESCOLA', StringType(), True),
 StructField('ENSINO', StringType(), True),
 StructField('TREINEIRO', StringType(), True),
 StructField('UF_PROVA', StringType(), True),
 StructField('PRE_C_NATURAIS', StringType(), True),
 StructField('PRE_C_HUMANAS', StringType(), True),
 StructField('PRE_LINGUAGENS', StringType(), True),
 StructField('PRE_MATEMATICA', StringType(), True),
 StructField('NOTA_C_NATURAIS', IntegerType(), True),
 StructField('NOTA_C_HUMANAS', IntegerType(), True),
 StructField('NOTA_LINGAGENS', IntegerType(), True),
 StructField('NOTA_MATEMATICA', IntegerType(), True),
 StructField('LINGUA_ESTR', StringType(), True),
 StructField('STATUS_RE

In [65]:
enem_refined.show(5)

+------------+----+------------+----+--------+------------+------------+------+------+---------+--------+--------------+-------------+--------------+--------------+---------------+--------------+--------------+---------------+-----------+--------------+----------+----------+----------+----------+----------+------------+
|   INSCRICAO| ANO|FAIXA_ETARIA|SEXO|COR_RACA|ST_CONCLUSAO|ANO_CONCLUIU|ESCOLA|ENSINO|TREINEIRO|UF_PROVA|PRE_C_NATURAIS|PRE_C_HUMANAS|PRE_LINGUAGENS|PRE_MATEMATICA|NOTA_C_NATURAIS|NOTA_C_HUMANAS|NOTA_LINGAGENS|NOTA_MATEMATICA|LINGUA_ESTR|STATUS_REDACAO|NOTA_COMP1|NOTA_COMP2|NOTA_COMP3|NOTA_COMP4|NOTA_COMP5|NOTA_REDACAO|
+------------+----+------------+----+--------+------------+------------+------+------+---------+--------+--------------+-------------+--------------+--------------+---------------+--------------+--------------+---------------+-----------+--------------+----------+----------+----------+----------+----------+------------+
|200006271946|2020|          11|  

Um enriquecimento que pode ser feito é a adição de uma coluna com região baseado na coluna de estado de aplicação da prova. Pra isso, será criado um DataFrame que contenha as referências para a adição da coluna.